In [1]:
#Importing Libraries
import requests
import pandas as pd
import csv
import shutil

#### Retention Rate

In [2]:
# Extract retention rate wrt US from https://currencyconvert.online/exchange-rates/usd/allcurrencies
def extracting_exchange_rate():
    url='https://currencyconvert.online/exchange-rates/usd/allcurrencies'
    response= requests.get(url)
    #print(response)
    #print(response.status_code)
    t= response.text
    v=t
    rate=[]
    while'text-left' in v:
        i= v.index('text-left\"><span')
        rate.append(v[i:i+50].split(">")[2].split("<")[0])
        v=v[i+1:]
    v=t
    currency=[]
    while'd-inline d-md-none' in v:
        i= v.index('d-inline d-md-none')
        currency.append(v[i+20:i+23])
        v=v[i+1:]
    exchange_rate= pd.DataFrame({'Currency': currency, "Exchange_Rate": rate})
    exchange_rate=exchange_rate.append([{'Currency':'USD','Exchange_Rate':1}])
    exchange_rate['Exchange_Rate']=pd.to_numeric(exchange_rate['Exchange_Rate'])
    return exchange_rate

In [8]:
extracting_exchange_rate().to_csv('Exchange_Rate.csv')

In [3]:
# Find out convertion rate of given countries
def currency_converter_rate(exchange_rate, currency_to,  currency_from='INR'):
    er= exchange_rate.set_index('Currency').Exchange_Rate.to_dict()
    return  er[currency_from]/er[currency_to]
    

In [4]:
# Find out the converted value 
def currency_converter(currency_to,  currency_from='INR', amount=1):
    exchange_rate= extracting_exchange_rate()
    rate= currency_converter_rate(exchange_rate, currency_to,  currency_from)
    return amount * rate

In [5]:
currency_converter('USD','INR',1)

79.327

#### Purchase Power Parity

In [7]:
# Downloads the ppp.csv table from https://data.oecd.org/conversion/purchasing-power-parities-ppp.htm
def download_PPP_chart():
    url = 'https://stats.oecd.org/sdmx-json/data/DP_LIVE/.PPP.../OECD?contentType=csv&detail=code&separator=comma&csv-lang=en'
    r = requests.get(url, verify=False,stream=True)
    if r.status_code!=200:
        print("Failure!!")
        exit()
    else:
        r.raw.decode_content = True
        with open("PurchasePowerParity.csv", 'wb') as f:
            shutil.copyfileobj(r.raw, f)
        print("Success")

#if __name__ == '__main__':
download_PPP_chart()

C:\Users\roshn\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'stats.oecd.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Success


In [10]:
# Reads the csv and converts it into cross tab format
def reading_ppp_file():
    download_PPP_chart()
    ppp= pd.read_csv('PurchasePowerParity.csv')
    ppp=pd.crosstab(index = ppp.LOCATION, columns = ppp.TIME, values = ppp.Value, aggfunc = 'mean').reset_index()
    return ppp

In [11]:
reading_ppp_file().to_csv('PPP_rate.csv')

C:\Users\roshn\anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'stats.oecd.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Success


In [ ]:
# Finds the ppp convertion rate between 2 countries
def ppp_convertion_rate(ppp, currency_to, currency_from='IND'):
    ppp_dict= ppp.set_index('LOCATION')[ppp.columns[-1]].to_dict()
    return ppp_dict[currency_to]/ppp_dict[currency_from]


In [ ]:
# Find out the converted ppp value 
def ppp_converter(currency_to,  currency_from='IND', amount=1):
    ppp= reading_ppp_file()
    rate= ppp_convertion_rate(ppp, currency_to,  currency_from)
    return amount * rate

In [ ]:
#ppp_converter( 'IND', 'USA', amount=1)